In [1]:
import cv2
import numpy as np
import math

In [73]:
def AngCal(image):
    black_image = np.copy(image)
    black_image = cv2.cvtColor(black_image, cv2.COLOR_GRAY2BGR)
    
    left_points = []
    right_points = []

    h, w = image.shape
    HEIGHT_HORIZON = 50

    ratio = 5
    
    for row in range(int(h/ratio)):
        line_row = image[row*ratio, :]
        for x, y in enumerate(line_row):
            if x == 0 and y > 210:
                break
            if y > 210:
                min_x = x
                left_points.append((min_x, row*ratio))
                cv2.circle(black_image, (np.int64(min_x), row*ratio), 5, (0,255,255), -1)
                break
        for x, y in enumerate(reversed(line_row)):
            if x == 0 and y > 210:
                break
            if y > 210:
                max_x = w - x
                right_points.append((max_x, row*ratio))
                cv2.circle(black_image, (np.int64(max_x), row*ratio), 5, (255,0,255), -1)
                break

    left_points = np.array(left_points)
    right_points = np.array(right_points)

    ## Finding Middle Lane
    middle_points = list()
    num_points = 6 # number of points
    angle = 0
    if(left_points.shape[0] < num_points and right_points.shape[0] < num_points):
        angle = 0
    elif(left_points.shape[0] >= num_points and right_points.shape[0] >= num_points):
        # print('Two Lines')
        # Finding left function
        x_lefts = list()
        y_lefts = list()
        for i in left_points:
            x_lefts.append(i[0])
            y_lefts.append(i[1])
        left_func = np.polyfit(x_lefts,y_lefts,2)
        # Finding right function
        x_rights = list()
        y_rights = list()
        for i in right_points:
            x_rights.append(i[0])
            y_rights.append(i[1])
        right_func = np.polyfit(x_rights,y_rights,2)
        for i in range(239, HEIGHT_HORIZON, -1):
            ######################################################################
            x_left = np.roots([left_func[0], left_func[1], left_func[2]-i])
            if left_func[0] > 0:
                x1 = np.min(x_left)
            else:
                x1 = np.max(x_left)
                
            ######################################################################
            x_right = np.roots([right_func[0], right_func[1], right_func[2]-i])
            if right_func[0] > 0:
                x2 = np.max(x_right)
            else:
                x2 = np.min(x_right)
                
            if ((np.abs(np.imag(x1)) < 1e-10) > 0 and (np.abs(np.imag(x2)) < 1e-10) > 0):
                cv2.circle(black_image, (np.int64(x1), i), 5, (255,0,0), -1)
                cv2.circle(black_image, (np.int64(x2), i), 5, (0,0,255), -1)
                middle_point = (x1+x2)/2
                cv2.circle(black_image, (np.int64(middle_point), i), 5, (0,255,0), -1)
                middle_points.append([np.int64(middle_point),i])
    ## Finding Only Right Lane
    elif left_points.shape[0]<num_points:
        # print('Only Right Lane')
        Right_Offset = -145
        x_rights = list()
        y_rights = list()
        for i in right_points:
            x_rights.append(i[0])
            y_rights.append(i[1])
        right_func = np.polyfit(x_rights,y_rights,2)
        for i in range(239, HEIGHT_HORIZON, -1):
            x_right = np.roots([right_func[0], right_func[1], right_func[2]-i])
            if right_func[0] > 0:
                x2 = np.max(x_right)
            else:
                x2 = np.min(x_right)
            
            if (np.abs(np.imag(x2)) < 1e-10) > 0:
                cv2.circle(black_image, (np.int64(x2), i), 5, (0,0,255), -1)
                #################################################
                middle_point = np.int64(x2+Right_Offset-0.55*(i-240+1))
                cv2.circle(black_image, (np.int64(middle_point), i), 5, (0,255,0), -1)
                middle_points.append([np.int64(middle_point),i])
    ## Finding Only Left Lane
    elif right_points.shape[0]<num_points:
        # print('Only Left Lane')
        Left_Offset = 145
        x_lefts = list()
        y_lefts = list()
        for i in left_points:
            x_lefts.append(i[0])
            y_lefts.append(i[1])
        left_func = np.polyfit(x_lefts,y_lefts,2)
        for i in range(239, HEIGHT_HORIZON, -1):
            x_left = np.roots([left_func[0], left_func[1], left_func[2]-i])
            if left_func[0]>0:
                x1 = np.min(x_left)
            else:
                x1 = np.max(x_left)
            
            if (np.abs(np.imag(x1)) < 1e-10) > 0:
                cv2.circle(black_image, (np.int64(x1), i), 5, (255,0,0), -1)
                ########################################
                middle_point = np.int64(x1+Left_Offset+0.55*(i-240+1))
                cv2.circle(black_image, (np.int64(middle_point), i), 5, (0,255,0), -1)
                middle_points.append([np.int64(middle_point),i])
    # Calculate Angle
    chosen_middle = 80
    # angle_thres = 3
    
    if len(middle_points)>0:
        ## Only < chosen middle
        if middle_points[0][1] < chosen_middle:
            x = middle_points[0][0]
            y = middle_points[0][1]
            angle = np.int64(np.arctan((159-x)/(239-y))*(180/np.pi))
            angle = np.int64(angle/2.3)
            # print(x, y)
            cv2.circle(black_image, (x, y), 5, (128, 128, 128), -1)
        ## Only > chosen middle
        elif middle_points[-1][1] > chosen_middle:
            x = middle_points[-1][0]
            y = middle_points[-1][1]
            angle = np.int64(np.arctan((159-x)/(239-y))*(180/np.pi))
            if(y > chosen_middle and y < chosen_middle + 23): # thres 1
                # print('Thres 1')
                angle = np.int64(angle/2)
            elif(y >= chosen_middle + 23 and y < chosen_middle + 23*2): # thres 2
                # print('Thres 2')
                angle = np.int64(angle/2)
            elif(y >= chosen_middle + 23*2 and y < chosen_middle + 23*2 + 22): # thres 3
                # print('Thres 3')
                angle = np.int64(angle/2)
            elif(y >= chosen_middle + 23*2+22 and y < chosen_middle + 23*2 + 22*2): # thres 4
                # print('Thres 4')
                angle = np.int64(angle/2)
            # print(x, y)
            cv2.circle(black_image, (x, y), 5, (128, 128, 128), -1)
        else:
            for i in sorted(middle_points, key=lambda x: x[1]):
                if i[1] == chosen_middle:
                    x = i[0]
                    y = i[1]
                    angle = np.int64(np.arctan((159-x)/(239-y))*(180/np.pi)) 
                    angle = np.int64(angle/2.3)
                    cv2.circle(black_image, (x, y), 5, (128, 128, 128), -1)
                    # print(x, y)
                    break
            
    # Send Image
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.5
    font_thickness = 3
    text_color = (0, 0, 255)  # Màu văn bản: Trắng
    text_position = (10, 20)  # Vị trí của văn bản: Góc trái phía trên
    cv2.putText(black_image, str(float(-angle)), text_position, font, font_scale, text_color, font_thickness)

    return float(-angle), black_image

In [74]:
import os
for i in os.listdir('./data/'):
    img = cv2.imread(f'./data/{i}', 0)
    img = cv2.resize(img, (320, 240))
    cv2.imshow('img', img)
    angle, result = AngCal(img)
    cv2.imshow('res', result)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

KeyboardInterrupt: 

In [75]:
cv2.destroyAllWindows()